In [18]:
import pandas as pd
fire_data = pd.read_csv(r'fire_data8423.csv')
fire_data = fire_data.dropna()

# Drop columns if they exist
columns_to_drop = ['CONT_DATE', '_id']
fire_data = fire_data.drop(columns=[col for col in columns_to_drop if col in fire_data.columns])
unpivot_weather = pd.read_csv(r"C:\Users\natal\Desktop\IRC\IRC-Group5\combined_df.csv")
#try merging with unpivoted index weather data, tall and skinny data

print(unpivot_weather['date'].dtype)
print(fire_data['ALARM_DATE'].dtype)


int64
object


In [22]:
#unpivot_weather.drop(['level_0'],inplace=True)

# Convert 'date' column to datetime
#unpivot_weather['date'] = pd.to_datetime(unpivot_weather['date'], format='%Y%m%d')
# Convert ALARM_DATE to datetime
#fire_data['ALARM_DATE'] = pd.to_datetime(fire_data['ALARM_DATE'], format='%d/%m/%Y')
unpivot_weather['date'] = pd.to_datetime(unpivot_weather['date'])
fire_data['ALARM_DATE'] = pd.to_datetime(fire_data['ALARM_DATE'])

# Filter rows between 1984-01-01 and 2023-12-31
unpivot_weather = unpivot_weather[(unpivot_weather['date'] >= '19840101') & (unpivot_weather['date'] <= '20231231')]
fire_data = fire_data[(fire_data['ALARM_DATE'] >= '19840101') & (fire_data['ALARM_DATE'] <= '20231231')]



pd.merge on column is too slow, try making id and date index now

In [23]:
# Rename columns to match index names before setting index
unpivot_weather.rename(columns={'id': 'weatherstn', 'date': 'ALARM_DATE'}, inplace=True)

# Set index before merging (Speeds up join operations)
unpivot_weather.set_index(['weatherstn', 'ALARM_DATE'], inplace=True)
fire_data.set_index(['weatherstn', 'ALARM_DATE'], inplace=True)

# Perform index-based merge
unpivotweather_fire_merge = unpivot_weather.merge(fire_data, left_index=True, right_index=True, how='left')

# Reset index if needed
unpivotweather_fire_merge.reset_index(inplace=True)


In [24]:
unpivotweather_fire_merge= pd.read_csv(r"C:\Users\natal\Desktop\IRC\unpivotweather_fire_merge8423.csv")

C:\Users\natal\AppData\Local\Temp\ipykernel_1256\3069563211.py:1: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  unpivotweather_fire_merge= pd.read_csv(r"C:\Users\natal\Desktop\IRC\unpivotweather_fire_merge8423.csv")


In [25]:
# Add a 'fire_occurred' column where 1 indicates fire occurred (fire_area is not NaN) and 0 indicates no fire
unpivotweather_fire_merge['fire_occurred'] = unpivotweather_fire_merge['Shape__Area'].notna().astype(int)


In [26]:
#drop index column
unpivotweather_fire_merged = unpivotweather_fire_merge.drop(['index'], axis=1)


2nd attempt: i am gonna split the data into 2 version
1 version for randomforestregressor (only the fire_area data, taking out dates without fire)
1 version for randomforestclassifier (everything, looking at the binary column, yes/no fire)

### Regression model: Random forest regressor

#### encoder: Category_encoders

In [27]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

# Generating the regression df by dropping rows with NaN in 'Shape__Area'
regression_ver = unpivotweather_fire_merged.dropna(subset=['Shape__Area'])

#####


# Assuming 'obs' is the categorical column and 'Shape__Area' is your target column
encoder = ce.TargetEncoder(cols=['obs'])  # Target encoding the 'obs' column

# Selecting the features for X (categorical 'obs' and numerical 'obs_value')
X = regression_ver[['obs', 'obs_value']]


# Fit and transform the encoder
X_encoded = encoder.fit_transform(X, regression_ver['GIS_ACRES'])  # You can encode 'obs' using target encoding


# Define the target variables (multiple columns if necessary)
Y = regression_ver[['GIS_ACRES', 'Shape__Length','Shape__Area']]  # Multiple target columns



In [28]:
#training and testing datasets split
from sklearn.model_selection import train_test_split
# Splitting the dataset into training and testing set (80/20)
X_train, X_test, Y_train, Y_test = train_test_split(X_encoded, Y, test_size = 0.2, random_state = 42)
# Initialize the RandomForestRegressor
model = RandomForestRegressor(n_estimators=50, random_state=42, oob_score=True)


model.fit(X_train,Y_train)


c:\Users\natal\anaconda3\Lib\site-packages\sklearn\utils\_array_api.py:245: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


RandomForestRegressor(n_estimators=50, oob_score=True, random_state=42)

In [29]:
# Make predictions
Y_pred = model.predict(X_test)
Y_pred
# Evaluate model performance
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")
print(f"OOB Score: {model.oob_score_}")


Mean Squared Error: 2.027829150998866e+16
R-squared: -0.017937530453420614
OOB Score: -0.040539321885986634
